# MIDI to RNN Input data treatment

This script was implemented to transform the pre-treated midi data to value arrays that can be served as input for the RNN training.

In [18]:
import pretty_midi
import numpy as np
import pandas as pd
import glob

## MIDI data attributes analysis

In [14]:
midi_files = glob.glob(r"..\midi\treated_midi\*")
midi_file = pretty_midi.PrettyMIDI(midi_files[0])

instrument = midi_file.instruments[0]
print(pretty_midi.program_to_instrument_name(instrument.program))

note = instrument.notes[0]
print(note)

Acoustic Grand Piano
Note(start=0.000000, end=0.136364, pitch=76, velocity=60)


Looking at the previous snippet, the Note class has four attributes:
* **start**: time when the note starts (in seconds)
* **end**: time when the note ends (in seconds)
* **pitch**: pitch of the note (can be associated with the piano key)
* **velocity**: information on the volume of the note - during pre-treatment all notes were set with velocity of 60.

We can disconsider the velocity information, as it has no relevant information for the structure of the song.

First, the data can be moved to an dictionary of dataframes, in which every single dataframe corresponds to a song.

In [21]:
data_dict = {}
for midi_file in midi_files:
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    song_dict = {
        "start": [],
        "end": [],
        "pitch": [],
    }
    for note in midi_data.instruments[0].notes:
        song_dict["start"].append(note.start)
        song_dict["end"].append(note.end)
        song_dict["pitch"].append(note.pitch)
    data_dict[midi_file.split("\\")[-1].split(".")[0]] = pd.DataFrame(song_dict)

dict_key = list(data_dict.keys())[0]
print(f'{dict_key}\n{"-"*40}\n{data_dict[dict_key]}')

160_EIGHTMEASURE_EXERCISES_NO_1
----------------------------------------
         start        end  pitch
0     0.000000   0.136364     76
1     0.136364   0.272727     79
2     0.272727   0.409091     77
3     0.000000   0.545455     60
4     0.000000   0.545455     67
..         ...        ...    ...
147  15.818182  15.954545     72
148  15.954545  16.090909     76
149  16.090909  16.227273     79
150  16.227273  16.363636     84
151  16.363636  16.636364     72

[152 rows x 3 columns]


In [16]:
min_pitch, max_pitch = 0 , 0

for midi_file in midi_files:
    midi_file = pretty_midi.PrettyMIDI(midi_file)
    notes = midi_file.instruments[0].notes
    for note in notes:
        min_pitch = min(min_pitch, note.pitch)
        max_pitch = max(max_pitch, note.pitch)

print(f'Pitch\nMin: {min_pitch}\tMax: {max_pitch}')

Pitch
Min: 0	Max: 105


As seen in previous snippet, the pitch of the notes vary from 0 to 105 in value, which can be translated from 8.18 Hz to 3520 Hz.
In a regular piano, the note of 8.18Hz is not achievable. The piano notes only start from 27.50Hz, which is the note A0, in MIDI pitch value of 21.
Having this in mind we can remove all notes under the pitch value of 21.

The max pitch value of 105 (or A7, 3520Hz) is valid for piano, so we don't need to trim in the high-pitch side.

In [ ]:
for i,midi_file in enumerate(midi_files):
    midi_file = pretty_midi.PrettyMIDI(midi_file)
    notes = midi_file.instruments[0].notes
    notes = [note for note in notes if note.pitch > 21]
    